In [63]:
from utils import inference_validator, reasoning, feature_engineering, additional_feature_engineering

In [64]:
import pickle, json
with open('saved/nimputer.pkl', 'rb') as f:
    nimputer = pickle.load(f)

with open('saved/cimputer.pkl', 'rb') as f:
    cimputer = pickle.load(f)

with open('saved/label_encoders.pkl', 'rb') as f:
    label_encoders = pickle.load(f)

    
with open('saved/inference_preprocessor.pkl', 'rb') as f:
    inference_preprocessor = pickle.load(f)

with open('saved/xgb_model.pkl', 'rb') as f:
    xgb = pickle.load(f)

with open('saved/before_feature.json', 'r') as f:
    before_columns = json.load(f)

In [65]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer

# Example user input
user_input = {
    'score_3': 0.5, 'score_4': np.nan, 'score_5': 0.7, 'score_6': np.nan,
    'risk_rate': 0.2, 'last_amount_borrowed': 1000, 'last_borrowed_in_months': 12,
    'credit_limit': 5000, 'income': np.nan, 'ok_since': 24, 'n_bankruptcies': 9,
    'n_defaulted_loans': 1, 'n_accounts': 3, 'n_issues': 2,
    'external_data_provider_credit_checks_last_year': 3,
    'external_data_provider_email_seen_before': 1, 'reported_income': 50000,
    'application_time_in_funnel': 5, 'external_data_provider_credit_checks_last_month': 0,
    'external_data_provider_fraud_score': 50, 'shipping_state': np.nan,
    'facebook_profile': True, 'state': np.nan, 'score_1': np.nan, 'score_2': np.nan, "real_state":'a'
}

# user_input = {
#     'score_3': 510, 'score_4': np.nan, 'score_5': 0.480, 'score_6': 92,
#     'risk_rate': 0.45, 'last_amount_borrowed': 600, 'last_borrowed_in_months': 20,
#     'credit_limit': 500, 'income': 60000, 'ok_since': 32, 'n_bankruptcies': 3.0,
#     'n_defaulted_loans': 0, 'n_accounts': 25.0, 'n_issues': 5.0,
#     'external_data_provider_credit_checks_last_year': 4.0,
#     'external_data_provider_email_seen_before': 5, 'reported_income': 50000,
#     'application_time_in_funnel': 900, 'external_data_provider_credit_checks_last_month': 5.0,
#     'external_data_provider_fraud_score': 250, 'shipping_state': np.nan,
#     'facebook_profile': True, 'state': np.nan, 'score_1': '1Rk8w4Ucd5yR3KcqZzLdow==', 'score_2': np.nan, "real_state": np.nan
# }

user_input = {
    'score_3': 800,  # High credit score
    'score_4': np.nan,  # Missing value
    'score_5': 0.850,  # High probability score
    'score_6': 98,  # High score
    'risk_rate': 0.10,  # Low risk rate
    'last_amount_borrowed': 200,  # Low amount borrowed
    'last_borrowed_in_months': 36,  # Long time since last borrowing
    'credit_limit': 10,  # High credit limit
    'income': 120000,  # High income
    'ok_since': 60,  # Long time since last issue
    'n_bankruptcies': 0.0,  # No bankruptcies
    'n_defaulted_loans': 0,  # No defaulted loans
    'n_accounts': 10.0,  # Moderate number of accounts
    'n_issues': 1.0,  # Few issues
    'external_data_provider_credit_checks_last_year': 1.0,  # Few credit checks
    'external_data_provider_email_seen_before': 10,  # Email seen many times (trusted)
    'reported_income': 120000,  # High reported income
    'application_time_in_funnel': 300,  # Short application time
    'external_data_provider_credit_checks_last_month': 0.0,  # No recent credit checks
    'external_data_provider_fraud_score': 100,  # Low fraud score
    'shipping_state': np.nan,  # Missing value
    'facebook_profile': False,  # Has a Facebook profile
    'state': np.nan,  # Missing value
    'score_1': '1Rk8w4Ucd5yR3KcqZzLdow==',  # Example encrypted score
    'score_2': np.nan,  # Missing value
    'real_state': np.nan  # Missing value
}

user_input = inference_validator(user_input)

user_df = pd.DataFrame([user_input])

# Ensure the user input has the same columns as the training data
user_df = user_df.reindex(columns=before_columns, fill_value=np.nan)

# Assuming you know which features each imputer was trained on
c_features = [f for f in cimputer.feature_names_in_ if f in user_df.columns]
n_features = [f for f in nimputer.feature_names_in_ if f in user_df.columns]


# Apply only if features exist
if c_features:
    user_df[c_features] = cimputer.transform(user_df[c_features])
if n_features:
    user_df[n_features] = nimputer.transform(user_df[n_features])

user_df = user_df.drop(columns=["target_default"])
# print("target_default" in user_df.columns)

# Apply the same label encoding to the user input
for col in c_features:
    if col in user_df.columns and col != "target_default":
        # Use the transform method of the fitted LabelEncoder
        user_df[col] = label_encoders[col].transform(user_df[col])

yo_features = [i for i in c_features + n_features if i != "target_default"]
user_df = user_df.reindex(columns=yo_features)

user_df = feature_engineering(user_df)
user_df = additional_feature_engineering(user_df)
# print(user_df)
user_processed = inference_preprocessor.transform(user_df)
print(user_df)


# Make prediction
prediction = xgb.predict(user_processed)
prediction_proba = xgb.predict_proba(user_processed)

print("Prediction:", prediction)
print(reasoning[str(prediction[0])])
print("Prediction Probability:", prediction_proba)
print(user_processed)
# 6 → 3 → 7 → 1 → 2 → 0 → 5 → 4

   score_1  score_2  facebook_profile  state  real_state  shipping_state  \
0        0       26                 0     47           1              23   

   score_3    score_4  score_5  score_6  ...  score_1_bin_x_score_2_bin  \
0    800.0  100.01795     0.85     98.0  ...                          0   

   score_1_bin_x_fraud_score_bin  score_2_bin_x_fraud_score_bin  \
0                              0                              0   

   state_x_real_state  state_x_shipping_state  state_real_state_avg_score_1  \
0                47_1                   47_23                           0.0   

   state_shipping_state_avg_score_2  score_1_sq  score_2_sq  \
0                              26.0           0         676   

   facebook_profile_sq  
0                    0  

[1 rows x 68 columns]
Prediction: [4]
Low-Medium Risk 
 This cluster has a low default rate (0.119) and shows good overall reliability. Borrowers in this cluster have taken moderate-sized loans (last_amount_borrowed: 0.064) 